In [ ]:
# # TensorFlow 설치
# %pip install tensorflow

# # PyTorch 설치 (기본 CPU 버전)
# %pip install torch torchvision torchaudio

#### 데이터 불러오기

In [ ]:
# # 데이터 불러오기 1 : 기존 수치형 데이터
# import pandas as pd
# url = "C:\\Users\\Playdata2\\Downloads\\tree_model_preprocessed.csv"
# df = pd.read_csv(url)
# df.head()

In [ ]:
# 데이터 불러오기 2 : 로그변환 수치형 데이터
import pandas as pd
url = "C:\\Users\\Playdata2\\Downloads\\re_log_model_preprocessed.csv"
df = pd.read_csv(url)
df.head()

In [ ]:
df.count().sum()

In [ ]:
df.shape

In [ ]:
df['churn'].value_counts()

#### 스케일링 별 데이터 생성

In [ ]:
# 스케일링 별 데이터 생성
# 1. 라이브러리 임포트
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

# 2. 입력(X), 타깃(y) 분리
X = df.drop('churn', axis=1)
y = df['churn']

# 3. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. 스케일링
# StandardScaler (기본), MinMaxScaler (정규화), RobustScaler (이상치 많을 때)
scaler = [StandardScaler(), MinMaxScaler(), RobustScaler()]

X_train_scaled = scaler[0].fit_transform(X_train)
X_train_MMS_scaled = scaler[1].fit_transform(X_train)
X_train_RS_scaled = scaler[2].fit_transform(X_train)

X_test_scaled = scaler[0].transform(X_test)
X_test_MMS_scaled = scaler[1].transform(X_test)
X_test_RS_scaled = scaler[2].transform(X_test)

In [ ]:
X_train.shape[1]

#### Torch

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

# 평가 지표 계산 함수
def compute_sequence_metrics(pred_orders, target_orders):
    y_true = []
    y_pred = []

    if isinstance(pred_orders[0], list):
        for gt, pred in zip(target_orders, pred_orders):
            if len(gt) != len(pred):
                continue
            y_true.extend(gt)
            y_pred.extend(pred)
    else:
        y_true = target_orders
        y_pred = pred_orders

    # Macro-average : 클래스별 성능 지표를 각각 계산 후 평균 -> 클래스별 데이터셋이 균등하게 분포되어 있을 때 적절
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)

    return f1, precision, recall



class Dataset_Module:
    def __init__(self, file_name):
        super().__init__()
        self.file_name = file_name

        df = pd.read_csv(f'../data/{self.file_name}')
        X = df.drop(columns=["churn", "remaining_contract", "is_tv_subscriber", "is_movie_package_subscriber"], axis=1)
        y = df['churn']

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(X, y, random_state=42, test_size=0.2,
                                                                              stratify=y)

    def get_train_dataset(self):
        train_dataset = Train_Dataset(self.X_train, self.y_train)
        val_dataset = Train_Dataset(self.X_val, self.y_val)

        return train_dataset, val_dataset


# PyTorch용 Dataset 정의
class Train_Dataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __len__(self):
        return len(self.data)

    # getter
    def __getitem__(self, idx):
        data = self.data.iloc[idx].to_numpy() # Pandas Series -> Numpy 배열
        label = self.label.iloc[idx] # int 형식의 값들 이므로 Numpy 배열 변환 X

        return data, label



# 다층 퍼셉트론 Multi-Layer Perceptron 신경망 정의
class MLPNet(nn.Module):
    def __init__(self, Cin):
        super().__init__()
        self.Cin = Cin

        self.layers = nn.Sequential(
            nn.Linear(Cin, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1) # 이진분류 출력 노드 1개
        )

    def forward(self, x):
        return self.layers(x)



def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    dataset = Dataset_Module(file_name='train.csv')
    train_dataset, val_dataset = dataset.get_train_dataset()

    # 입력 특성 수 확인
    Cin = train_dataset.__getitem__(0)[0].shape
    print(f"num Feature : {Cin}")

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)

    model = MLPNet(Cin[0])
    criterion = nn.BCEWithLogitsLoss() # 이진분류 손실함수
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)

    train_epoch = 100
    train_losses = []
    val_losses = []
    val_accuracies = []

    # 최고 성능 기록 위한 dict
    best_metrics = {'f1': 0, 'precision': 0, 'recall': 0, 'accuracy': 0}

    for epoch in range(train_epoch):
        # 학습
        model.train()
        train_loss = 0.0
        train_iter = tqdm(train_loader, desc=f"[Epoch {epoch + 1}/{train_epoch}] Training")

        for batch_data, batch_label in train_iter:
            batch_data, batch_label = batch_data.to(device).float(), batch_label.to(device).float().unsqueeze(-1)
            optimizer.zero_grad()
            outputs = model(batch_data)
            loss = criterion(outputs, batch_label)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_iter.set_postfix(loss=loss.item())

        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # 검증
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch_data, batch_label in tqdm(val_loader, desc=f"[Epoch {epoch + 1}/{train_epoch}] Validation"):
                batch_data, batch_label = batch_data.to(device).float(), batch_label.to(device).float().unsqueeze(-1)

                outputs = model(batch_data)
                loss = criterion(outputs, batch_label)
                val_loss += loss.item()
                outputs = F.sigmoid(outputs)
                preds = (outputs > 0.5).float()

                correct += (preds == batch_label).sum().item()
                total += batch_label.size(0)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(batch_label.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        val_accuracy = 100 * correct / total
        val_accuracies.append(val_accuracy)

        # 평가지표 계산
        val_f1, val_precision, val_recall = compute_sequence_metrics(all_preds, all_labels)

        print(f"Train Loss : {avg_train_loss:.4f} || Valid Loss : {avg_val_loss:.4f} | Valid Accuracy : {val_accuracy:.4f}%")
        print(f"Epoch {epoch + 1} - F1 : {val_f1:.4f}, Precision : {val_precision:.4f}, Recall : {val_recall:.4f}")

        # 최고 성능 갱신 시 모델 저장
        if val_accuracy > best_metrics['accuracy']:
            best_metrics['f1'] = val_f1
            best_metrics['precision'] = val_precision
            best_metrics['recall'] = val_recall
            best_metrics['accuracy'] = val_accuracy
            torch.save(model.state_dict(), '../best_model.pt')  # 모델 파라미터 저장

    # 최종능 평가 지표 출력
    print("\n=== 최종 평가 지표 (Validation Best Accuracy 기준) ===")
    print(f"F1 Score     : {best_metrics['f1']:.4f}")
    print(f"Precision    : {best_metrics['precision']:.4f}")
    print(f"Recall       : {best_metrics['recall']:.4f}")
    print(f"Accuracy (%) : {best_metrics['accuracy']:.2f}%")



if __name__ == '__main__':
    main()


# === 최종 평가 지표 (Validation Best Accuracy 기준) ===
# F1 Score     : 0.7327
# Precision    : 0.7386
# Recall       : 0.7308
# Accuracy (%) : 73.95%

#### TensorFlow

In [ ]:
# 0. 라이브러리 임포트
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report

##### 기존 수치형 데이터 와 로그변환 데이터 성능 비교

In [ ]:
# 기존 수치형 데이터 와 로그변환 데이터 비교용 모델
# StandardScaler, layer 2, 64-32-1, relu, sigmoid, adam, binary_crossentropy
# 1. 모델 정의
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# 2. 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 3. 학습
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, verbose=2)

# 4. 평가
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"\n Test Accuracy: {test_acc:.4f}")

# 4. 예측
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

# 6. 성능 조회
print(f"\n=== Log Data ===")
# print(f"\n=== Num Data ===")
print(classification_report(y_test, y_pred))


In [ ]:
# 로그변환 모델이 0.1만큼 좋았다.

##### 레이어층 2, 뉴런조합별 성능 출력

In [ ]:
# 64-32-1
# StandardScaler, layer 2, 64-32-1, relu, sigmoid, adam, binary_crossentropy
# 1. 모델 정의
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# 2. 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 3. 학습
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, verbose=2)

# 4. 평가
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"\n Test Accuracy: {test_acc:.4f}")

# 4. 예측
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

# 6. 성능 조회
print(f"\n=== 64-32-1 ===")
print(classification_report(y_test, y_pred))

In [ ]:
# 32-16-1
# 기존 수치형 데이터 와 로그변환 데이터 비교용 모델
# StandardScaler, layer 2, 32-16-1, relu, sigmoid, adam, binary_crossentropy
# 1. 모델 정의
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# 2. 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 3. 학습
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, verbose=2)

# 4. 평가
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"\n Test Accuracy: {test_acc:.4f}")

# 4. 예측
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

# 6. 성능 조회
print(f"\n=== 32-16-1 ===")
print(classification_report(y_test, y_pred))

In [ ]:
# 16-8-1
# 기존 수치형 데이터 와 로그변환 데이터 비교용 모델
# StandardScaler, layer 2, 16-8-1, relu, sigmoid, adam, binary_crossentropy
# 1. 모델 정의
model = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# 2. 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 3. 학습
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, verbose=2)

# 4. 평가
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"\n Test Accuracy: {test_acc:.4f}")

# 4. 예측
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

# 6. 성능 조회
print(f"\n=== 16-8-1 ===")
print(classification_report(y_test, y_pred))

##### 레이어층 3, 뉴런조합별 성능 출력
- 피라미드형
- 늘렸다 줄이기
- 줄였다 늘리기
과적합 일어날시 Drop out을 사용하지만
Drop out적용해서 성능 수치 올라가는 경우도 있다고 하니, Drop out사용하기

##### 피라미드형

In [ ]:
# 128 -> 64 -> 32 -> 1
# 1. 모델 정의
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# 2. 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 3. 학습
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, verbose=2)

# 4. 평가
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"\n Test Accuracy: {test_acc:.4f}")

# 4. 예측
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

# 6. 성능 조회
print(f"\n=== 128 -> 64 -> 32 -> 1 ===")
print(classification_report(y_test, y_pred))

In [ ]:
# 64  -> 32 -> 16 -> 1
# 1. 모델 정의
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# 2. 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 3. 학습
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, verbose=2)

# 4. 평가
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"\n Test Accuracy: {test_acc:.4f}")

# 4. 예측
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

# 6. 성능 조회
print(f"\n=== 64-32-1 ===")
print(classification_report(y_test, y_pred))

In [ ]:
# 32 -> 16 -> 8 -> 1
# 1. 모델 정의
model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# 2. 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 3. 학습
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, verbose=2)

# 4. 평가
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"\n Test Accuracy: {test_acc:.4f}")

# 4. 예측
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

# 6. 성능 조회
print(f"\n=== 64-32-1 ===")
print(classification_report(y_test, y_pred))